In [1]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import folium 
print('Libraries imported.')

Libraries imported.


In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
Postalcd = df['Postal Code'].tolist()
Borough = df['Borough'].tolist()
Neighborhood = df['Neighborhood'].tolist()
dftor= pd.DataFrame(list(zip(Postalcd, Borough, Neighborhood )), 
               columns =['Postal Code', 'Borough','Neighborhood'])
dftor.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
dftor = dftor.dropna()
dftor.head(20)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
dfgeo=pd.read_csv('http://cocl.us/Geospatial_data')
dftoronto = pd.merge(dftor, dfgeo, on='Postal Code', how ='outer')
dftoronto.head(100)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [18]:
dftor2=dftoronto.drop('Neighborhood', axis=1).join(dftoronto['Neighborhood'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Neighborhood'))

In [19]:
dftor2.head(20)

,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M3A,North York,43.753259,-79.329656,Parkwoods
1,M4A,North York,43.725882,-79.315572,Victoria Village
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront
3,M6A,North York,43.718518,-79.464763,Lawrence Manor
3,M6A,North York,43.718518,-79.464763,Lawrence Heights
4,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park
4,M7A,Downtown Toronto,43.662301,-79.389494,Ontario Provincial Government
5,M9A,Etobicoke,43.667856,-79.532242,Islington Avenue
5,M9A,Etobicoke,43.667856,-79.532242,Humber Valley Village


In [21]:
df_tor=dftor2[dftor2['Borough'].str.contains('Toronto')]
df_tor.head(20)

,Postal Code,Borough,Latitude,Longitude,Neighborhood
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront
4,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park
4,M7A,Downtown Toronto,43.662301,-79.389494,Ontario Provincial Government
9,M5B,Downtown Toronto,43.657162,-79.378937,Garden District
9,M5B,Downtown Toronto,43.657162,-79.378937,Ryerson
15,M5C,Downtown Toronto,43.651494,-79.375418,St. James Town
19,M4E,East Toronto,43.676357,-79.293031,The Beaches
20,M5E,Downtown Toronto,43.644771,-79.373306,Berczy Park
24,M5G,Downtown Toronto,43.657952,-79.387383,Central Bay Street


In [22]:
CLIENT_ID = 'MOFCLWFI5MOZ0AH2JJBAW3LZ3EVG2J1GDHAOMMRVCMGBO1OS' # your Foursquare ID
CLIENT_SECRET = '21XYGON4S4GRID4CEOLLUSSVGVD0U1F4TL0WLU0PV2AKVJRA' # your Foursquare Secret
VERSION = '20200525' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MOFCLWFI5MOZ0AH2JJBAW3LZ3EVG2J1GDHAOMMRVCMGBO1OS
CLIENT_SECRET:21XYGON4S4GRID4CEOLLUSSVGVD0U1F4TL0WLU0PV2AKVJRA


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
                       lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:

toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )


Regent Park
 Harbourfront
Queen's Park
 Ontario Provincial Government
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
North Toronto West
 Lawrence Park
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
R

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(3163, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.021978,0.0,0.0,...,0.0,0.000000,0.00,0.010989,0.000000,0.000000,0.000000,0.0,0.010989,0.000000
1,Bathurst Quay,0.0,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,Cabbagetown,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Chinatown,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.00,0.034483,0.000000,0.051724,0.017241,0.0,0.000000,0.000000
4,Deer Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.0,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,The Annex,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.00,0.045455,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
73,The Beaches,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
74,The Danforth West,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.0,0.0,...,0.0,0.023810,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.023810
75,Toronto Dominion Centre,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.0,0.0,...,0.0,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000


In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.05
2      Restaurant  0.04
3           Hotel  0.03
4  Clothing Store  0.03


---- Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3          Boutique  0.07
4           Airport  0.07


---- Cabbagetown----
                venue  freq
0         Coffee Shop  0.07
1  Chinese Restaurant  0.05
2                Café  0.05
3         Pizza Place  0.05
4              Bakery  0.05


---- Chinatown----
                   venue  freq
0                   Café  0.09
1            Coffee Shop  0.07
2     Mexican Restaurant  0.05
3           Dessert Shop  0.05
4  Vietnamese Restaurant  0.05


---- Deer Park----
                venue  freq
0                 Pub  0.12
1         Coffee Shop  0.12
2                Bank  0.12
3  Light Rail Station  0.06
4          Bagel Shop  0.06


---- Design Exchange----
                venue  freq
0         Coffe

               venue  freq
0         Playground  0.33
1        Summer Camp  0.33
2         Restaurant  0.33
3      Movie Theater  0.00
4  Martial Arts Dojo  0.00


----North Toronto West----
                 venue  freq
0       Clothing Store  0.18
1          Coffee Shop  0.09
2          Yoga Studio  0.05
3  Sporting Goods Shop  0.05
4                 Park  0.05


----Parkdale----
                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2                Movie Theater  0.07
3  Eastern European Restaurant  0.07
4                          Bar  0.07


----Queen's Park----
                 venue  freq
0          Coffee Shop  0.22
1          Yoga Studio  0.03
2                  Gym  0.03
3   College Auditorium  0.03
4  Fried Chicken Joint  0.03


----Regent Park----
            venue  freq
0     Coffee Shop  0.16
1             Pub  0.07
2            Park  0.07
3          Bakery  0.07
4  Breakfast Spot  0.04


----Richmond----
            

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Gym,Hotel,Thai Restaurant,Steakhouse,Concert Hall
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Plane,Boutique,Bar,Airport Food Court
2,Cabbagetown,Coffee Shop,Café,Restaurant,Italian Restaurant,Pub,Bakery,Chinese Restaurant,Pizza Place,Gym / Fitness Center,Pet Store
3,Chinatown,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bakery,Dessert Shop,Bar,Vegetarian / Vegan Restaurant,Gaming Cafe,Grocery Store
4,Deer Park,Coffee Shop,Bank,Pub,Fried Chicken Joint,Light Rail Station,Supermarket,Vietnamese Restaurant,Liquor Store,Bagel Shop,American Restaurant


In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 0, 1, 1, 1, 2, 0])

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
2,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
4,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park,1,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Restaurant,Café,Park
4,M7A,Downtown Toronto,43.662301,-79.389494,Ontario Provincial Government,1,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Restaurant,Café,Park
9,M5B,Downtown Toronto,43.657162,-79.378937,Garden District,1,Clothing Store,Coffee Shop,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Tea Room


In [39]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [43]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters